### Carga de Datos

In [24]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!unzip udea*.zip > /dev/null

In [5]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


In [61]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)

shape of loaded dataframe (692500, 21)


### Análisis inicial

In [34]:
z.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [35]:
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 21 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   ID                           692500 non-null  int64  
 1   PERIODO_ACADEMICO            692500 non-null  int64  
 2   E_PRGM_ACADEMICO             692500 non-null  object 
 3   E_PRGM_DEPARTAMENTO          692500 non-null  object 
 4   E_VALORMATRICULAUNIVERSIDAD  686213 non-null  object 
 5   E_HORASSEMANATRABAJA         661643 non-null  object 
 6   F_ESTRATOVIVIENDA            660363 non-null  object 
 7   F_TIENEINTERNET              665871 non-null  object 
 8   F_EDUCACIONPADRE             669322 non-null  object 
 9   F_TIENELAVADORA              652727 non-null  object 
 10  F_TIENEAUTOMOVIL             648877 non-null  object 
 11  E_PRIVADO_LIBERTAD           692500 non-null  object 
 12  E_PAGOMATRICULAPROPIO        686002 non-null  object 
 13 

In [62]:
#Eliminar las columnas que no aportan información
def drop_columns(df, columns = ["ID","PERIODO_ACADEMICO","E_PRGM_ACADEMICO","E_PRGM_DEPARTAMENTO"]):
  return df.drop(columns, axis=1)


#Inputar valores faltantes
def input_data(df):
  new_df = df.copy()
  new_categories = [                      #Columnas en las cuales se asignará una nueva categoría "Sin información" a los valores faltantes
      "E_VALORMATRICULAUNIVERSIDAD",
      "E_HORASSEMANATRABAJA",
      "F_EDUCACIONPADRE",
      "F_EDUCACIONMADRE"
  ]
  related_columns = [                     #Estas columnas se imputarán según los valores de otras columnas relacionadas ("E_VALORMATRICULAUNIVERSIDAD" y "F_ESTRATOVIVIENDA")
      "F_ESTRATOVIVIENDA",                #usando la moda de la columna a imputar según los valores de las 2 columnas anteriores.
      "F_TIENEINTERNET",
      "F_TIENELAVADORA",
      "F_TIENEAUTOMOVIL",
      "E_PAGOMATRICULAPROPIO",
      "F_TIENEINTERNET.1",
      "F_TIENECOMPUTADOR",
  ]
  for column in new_categories:
    new_df[column] = new_df[column].fillna("Sin información")
  for column in related_columns:
    if (column == "F_ESTRATOVIVIENDA"):
      new_df[column] = (
          new_df.groupby('E_VALORMATRICULAUNIVERSIDAD')[column]
          .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'Estrato 3'))
      )
      continue
    new_df[column] = (
    new_df.groupby(['E_VALORMATRICULAUNIVERSIDAD', 'F_ESTRATOVIVIENDA'])[column]
    .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No'))
    )
  return new_df

In [63]:
#Esta función recibe 3 listas, en cada posicion respectivamente, el nombre de una columna categorica ordinal, un diccionario para mapear la columna a un valor entero y el nombre de la nueva columna codificada
def ordinal_enconding(df, columns_to_input, new_columns_names, ordinals_dicts):
  new_df = df.copy()
  for i in range(len(columns_to_input)):
    new_df[new_columns_names[i]] = new_df[columns_to_input[i]].map(ordinals_dicts[i])
  new_df = new_df.drop(columns = columns_to_input, axis=1)
  return new_df


def processing(df, columns_to_input, new_columns_names, ordinals_dicts):
  new_df = df.copy()
  new_df = drop_columns(new_df)
  new_df = input_data(new_df)
  new_df = ordinal_enconding(new_df, columns_to_input, new_columns_names, ordinals_dicts)
  return new_df

columns_to_input = [
    "E_VALORMATRICULAUNIVERSIDAD",
    "E_HORASSEMANATRABAJA",
    "F_ESTRATOVIVIENDA",
    "F_EDUCACIONPADRE",
    "F_EDUCACIONMADRE",
    "F_TIENEINTERNET",
    "F_TIENELAVADORA",
    "F_TIENEAUTOMOVIL",
    "E_PRIVADO_LIBERTAD",
    "E_PAGOMATRICULAPROPIO",
    "F_TIENECOMPUTADOR",
    "F_TIENEINTERNET.1"
]

new_columns_names = [
    "CATEGORIA_VALOR_MATRICULA",
    "CATEGORIA_HORAS_TRABAJO",
    "CATEGORIA_ESTRATO",
    "CATEGORIA_EDUCACION_PADRE",
    "CATEGORIA_EDUCACION_MADRE",
    "CATEGORIA_INTERNET",
    "CATEGORIA_LAVADORA",
    "CATEGORIA_AUTO",
    "CATEGORIA_LIBERTAD",
    "CATEGORIA_PAGOMATRICULAPROPIO",
    "CATEGORIA_COMPUTADOR",
    "CATEGORIA_INTERNET.1"
]


ordinal_dicts = [
    {'Entre 5.5 millones y menos de 7 millones': 9,
 'Entre 2.5 millones y menos de 4 millones': 8,
 'Entre 4 millones y menos de 5.5 millones': 7,
 'Más de 7 millones': 6,
 'Entre 1 millón y menos de 2.5 millones': 5,
 'Entre 500 mil y menos de 1 millón': 4,
 'Menos de 500 mil': 3,
 'No pagó matrícula': 2,
 'Sin información': 1
     },
    {
  'Menos de 10 horas':2,
 '0':1,
 'Más de 30 horas':5,
 'Entre 21 y 30 horas':4,
 'Entre 11 y 20 horas':3,
 'Sin información':0
},
    {
    'Estrato 3':3,
 'Estrato 4':4,
 'Estrato 5':5,
 'Estrato 2':2,
 'Estrato 1':1,
 'Estrato 6':6,
 'Sin Estrato':0
},
    {
    'Técnica o tecnológica incompleta':8,
       'Técnica o tecnológica completa':9,
       'Secundaria (Bachillerato) completa':7,
       'No sabe':2,
       'Primaria completa':5,
       'Educación profesional completa':11,
       'Educación profesional incompleta':10,
        'Primaria incompleta':4,
       'Postgrado':12,
       'Secundaria (Bachillerato) incompleta':6,
       'Ninguno':3,
       'No Aplica':1,
       'Sin información':0
},
    {
    'Técnica o tecnológica incompleta':8,
       'Técnica o tecnológica completa':9,
       'Secundaria (Bachillerato) completa':7,
       'No sabe':2,
       'Primaria completa':5,
       'Educación profesional completa':11,
       'Educación profesional incompleta':10,
        'Primaria incompleta':4,
       'Postgrado':12,
       'Secundaria (Bachillerato) incompleta':6,
       'Ninguno':3,
       'No Aplica':1,
       'Sin información':0
}
    ,{"Si":1, "No":0},{"Si":1, "No":0},{"Si":1, "No":0},{"S":1, "N":0},{"Si":1, "No":0},{"Si":1, "No":0},{"Si":1, "No":0}
]






En la celda anterior se definen y agrupan distintas funciones para realizar el mismo pre-proceso para train y test.

Se definen funciones para:


*   Eliminar columnas (predefinidas)
*   Imputar faltantes de acuerdo a cada columna
*   Codificación
*   Aplicar todo al dataset (train/test)

Para la codificación se definen manualmente los diccionarios para mapear, de forma que se garantiza un valor ordenado y coherente con las distintas categorías.



In [64]:
z = processing(z, columns_to_input, new_columns_names, ordinal_dicts)

In [65]:
z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   RENDIMIENTO_GLOBAL             692500 non-null  object 
 1   INDICADOR_1                    692500 non-null  float64
 2   INDICADOR_2                    692500 non-null  float64
 3   INDICADOR_3                    692500 non-null  float64
 4   INDICADOR_4                    692500 non-null  float64
 5   CATEGORIA_VALOR_MATRICULA      692500 non-null  int64  
 6   CATEGORIA_HORAS_TRABAJO        692500 non-null  int64  
 7   CATEGORIA_ESTRATO              692500 non-null  int64  
 8   CATEGORIA_EDUCACION_PADRE      692500 non-null  int64  
 9   CATEGORIA_EDUCACION_MADRE      692500 non-null  int64  
 10  CATEGORIA_INTERNET             692500 non-null  int64  
 11  CATEGORIA_LAVADORA             692500 non-null  int64  
 12  CATEGORIA_AUTO                

In [66]:
#Separación y codificación de la columna target

ytr = z["RENDIMIENTO_GLOBAL"].map({
    'medio-alto':3,
    'bajo':1,
    'alto':4,
    'medio-bajo':2
})

z.drop("RENDIMIENTO_GLOBAL", axis=1, inplace=True)

In [67]:
ytr

,RENDIMIENTO_GLOBAL
0,3
1,1
2,1
3,4
4,2
...,...
692495,3
692496,1
692497,2
692498,1


In [68]:
test = pd.read_csv("./test.csv")

In [69]:
ids = test["ID"]

In [70]:
ids

,ID
0,550236
1,98545
2,499179
3,782980
4,785185
...,...
296781,496981
296782,209415
296783,239074
296784,963852


In [71]:
test = processing(test, columns_to_input, new_columns_names, ordinal_dicts)

In [72]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 16 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   INDICADOR_1                    296786 non-null  float64
 1   INDICADOR_2                    296786 non-null  float64
 2   INDICADOR_3                    296786 non-null  float64
 3   INDICADOR_4                    296786 non-null  float64
 4   CATEGORIA_VALOR_MATRICULA      296786 non-null  int64  
 5   CATEGORIA_HORAS_TRABAJO        296786 non-null  int64  
 6   CATEGORIA_ESTRATO              296786 non-null  int64  
 7   CATEGORIA_EDUCACION_PADRE      296786 non-null  int64  
 8   CATEGORIA_EDUCACION_MADRE      296786 non-null  int64  
 9   CATEGORIA_INTERNET             296786 non-null  int64  
 10  CATEGORIA_LAVADORA             296786 non-null  int64  
 11  CATEGORIA_AUTO                 296786 non-null  int64  
 12  CATEGORIA_LIBERTAD            

In [73]:
#Guardar datasets procesados para entrenar otros modelos.

z.to_csv("processed_train.csv", index=False)
test.to_csv("processed_test.csv", index=False)
ytr.to_csv("ytr.csv", index=False)

In [74]:
from sklearn.ensemble import RandomForestClassifier

In [75]:
modelo = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)

In [76]:
modelo.fit(z, ytr)

RandomForestClassifier(n_estimators=200, n_jobs=-1, random_state=42)

In [77]:
y_pred = modelo.predict(test)

In [78]:
submit = pd.DataFrame({
    "ID": ids,
    "RENDIMIENTO_GLOBAL": y_pred
})

In [79]:
submit["RENDIMIENTO_GLOBAL"] = submit["RENDIMIENTO_GLOBAL"].map({
    3:'medio-alto',
    1:'bajo',
    4:'alto',
    2:'medio-bajo'
})

In [80]:
submit.to_csv("submit1.csv", index=False)